In [1]:
using ITensors 
using Plots
using Measurements   
using DelimitedFiles 

In [3]:
# lattice_large - lattice_small = 2 
function GrossNeveu(L::Int64,N::Int64,m::Float64,G²::Float64)
    l = L-2;
    sites = siteinds("Fermion",L)
    ampoₗ = OpSum()
    ampoₛ = OpSum()
    # Large Lattice 
    for j in 1:L-N
        ampoₗ -= 1im,"C",j,"Cdag",j+N
        ampoₗ += 1im,"C",j+N,"Cdag",j
    end
    for j in 1:N:L-N
        ampoₗ += (G²/2),"C * Cdag",j
    end
    for j in 1:L
        ampoₗ += (m*(-1)^j),"C * Cdag",j
    end
    Hₗ = MPO(ampoₗ, sites)
    ψₗ₀ = randomMPS(sites,100)
    ######################################
    ## small lattice 
    for j in 1:l-N
        ampoₛ -= 1im,"C",j,"Cdag",j+N
        ampoₛ += 1im,"C",j+N,"Cdag",j
    end
    for j in 1:N:l-N
        ampoₛ += (G²/2),"C * Cdag",j
    end
    for j in 1:l
        ampoₛ += (m*(-1)^j),"C * Cdag",j 
    end
    Hₛ = MPO(ampoₛ,sites[1:l])
    ψₛ₀ = randomMPS(sites[1:l],100)
#     sweep = Sweeps(sw)
#     setmaxdim!(sweep,repeat([1024],sw)...)
#     setcutoff!(sweep,1E-10)
    sweep = Sweeps([
         "maxdim" "mindim" "cutoff" 
          100      10       1e-16    
          200      10      1e-16    
          500      20      1e-18    
          1000     30      1e-20    
          1024     50      1e-22
         ])
#     sweep = Sweeps([
#          "maxdim" "mindim" "cutoff" 
#           1024      10       1e-16    
#           1024      10      1e-16    
#           1024      20      1e-18    
#           1000     30      1e-20    
#           1024     50      1e-22
#          ])
    eₗ₀,ψₗ = dmrg(Hₗ,ψₗ₀,sweep,outputlevel=1)
    eₛ₀,ψₛ = dmrg(Hₛ,ψₛ₀,sweep,outputlevel=1)
    # error computation
    nrm = norm(inner(Hₛ,ψₛ,Hₛ,ψₛ) - inner(ψₛ',Hₛ,ψₛ)^2)
    #  Overlap computation
    v1 = ITensor(sites[l+1]) 
    for j in 1:size(v1)[1]
        v1[j] = 1/sqrt(2)
    end
    v2 = ITensor(sites[L])
    for j in 1:size(v2)[1]
        v2[j] = 1/sqrt(2)
    end
    v = ITensor(1.0)
    for j in 1:l
        v = v*dag(ψₛ[j])*ψₗ[j]
    end
    v = v*v1*ψₗ[l+1]
    v = v*v2*ψₗ[L]
    return inner(v,v), norm(v), nrm 
end

GrossNeveu (generic function with 1 method)

In [70]:
result_inner, result_norm, result_error = nothing, nothing, nothing

(nothing, nothing, nothing)

In [71]:
# L::Integer, N::Integer,G²::Float64,m::Float64,sw::Integer
result_inner = Vector{ComplexF64}();
result_norm = Vector{ComplexF64}();
result_error = Vector{ComplexF64}();

In [72]:
# GrossNeveu(L,N,m,G²)
for j in 4:30
    println(j)
    nnr , nrm, err = GrossNeveu(j,1,0.5,1.5); # change BD --> 100
    push!(result_inner, nnr);
    push!(result_norm, nrm);
    push!(result_error, err);
end 

4
After sweep 1 energy=-1.2215904375200788  maxlinkdim=4 maxerr=0.00E+00 time=0.007
After sweep 2 energy=-1.222261505616916  maxlinkdim=4 maxerr=0.00E+00 time=0.008
After sweep 3 energy=-1.222274095051289  maxlinkdim=4 maxerr=0.00E+00 time=0.011
After sweep 4 energy=-1.2222743317733857  maxlinkdim=4 maxerr=0.00E+00 time=0.011
After sweep 5 energy=-1.2222743362254527  maxlinkdim=4 maxerr=0.00E+00 time=0.010
After sweep 1 energy=-0.6309728763025335  maxlinkdim=2 maxerr=0.00E+00 time=0.004
After sweep 2 energy=-0.6327815367368659  maxlinkdim=2 maxerr=0.00E+00 time=0.003
After sweep 3 energy=-0.6327822182807987  maxlinkdim=2 maxerr=0.00E+00 time=0.003
After sweep 4 energy=-0.632782218537222  maxlinkdim=2 maxerr=0.00E+00 time=0.003
After sweep 5 energy=-0.6327822185373182  maxlinkdim=2 maxerr=0.00E+00 time=0.003
5
After sweep 1 energy=-1.8366141793365403  maxlinkdim=4 maxerr=0.00E+00 time=0.016
After sweep 2 energy=-1.8846934405569347  maxlinkdim=4 maxerr=0.00E+00 time=0.020
After sweep 3 e

14
After sweep 1 energy=-4.641464164097487  maxlinkdim=100 maxerr=1.23E-07 time=1.265
After sweep 2 energy=-4.653574233662986  maxlinkdim=125 maxerr=5.83E-17 time=0.820
After sweep 3 energy=-4.653957700836876  maxlinkdim=126 maxerr=9.48E-19 time=0.838
After sweep 4 energy=-4.653961388027911  maxlinkdim=123 maxerr=5.98E-21 time=0.862
After sweep 5 energy=-4.653961394680181  maxlinkdim=124 maxerr=9.18E-23 time=0.865
After sweep 1 energy=-3.9768968241869365  maxlinkdim=64 maxerr=0.00E+00 time=0.550
After sweep 2 energy=-3.9838789839068123  maxlinkdim=63 maxerr=1.02E-18 time=0.216
After sweep 3 energy=-3.9838923764897105  maxlinkdim=62 maxerr=8.60E-19 time=0.167
After sweep 4 energy=-3.983892377353758  maxlinkdim=60 maxerr=7.92E-21 time=0.261
After sweep 5 energy=-3.9838923773538157  maxlinkdim=50 maxerr=8.52E-23 time=0.361
15
After sweep 1 energy=-5.280093932990237  maxlinkdim=100 maxerr=1.91E-06 time=1.212
After sweep 2 energy=-5.293084107222782  maxlinkdim=128 maxerr=0.00E+00 time=1.597

After sweep 4 energy=-7.3701566852116045  maxlinkdim=905 maxerr=9.87E-21 time=72.509
After sweep 5 energy=-7.370157029478494  maxlinkdim=874 maxerr=9.89E-23 time=82.783
24
After sweep 1 energy=-8.032240386165208  maxlinkdim=100 maxerr=4.05E-04 time=4.829
After sweep 2 energy=-8.060047195776303  maxlinkdim=200 maxerr=9.37E-11 time=14.749
After sweep 3 energy=-8.063537267822255  maxlinkdim=500 maxerr=3.01E-14 time=54.222
After sweep 4 energy=-8.063773717963958  maxlinkdim=1000 maxerr=4.63E-17 time=213.176
After sweep 5 energy=-8.063780975865503  maxlinkdim=1024 maxerr=8.04E-18 time=271.230
After sweep 1 energy=-7.374152531608019  maxlinkdim=100 maxerr=3.50E-04 time=2.887
After sweep 2 energy=-7.384720833582692  maxlinkdim=200 maxerr=8.48E-12 time=8.134
After sweep 3 energy=-7.385044279476936  maxlinkdim=500 maxerr=1.69E-16 time=30.542
After sweep 4 energy=-7.385058369681081  maxlinkdim=1000 maxerr=7.63E-20 time=124.108
After sweep 5 energy=-7.38505908610126  maxlinkdim=1024 maxerr=2.18E-

In [73]:
result_inner2, result_norm2, result_error2 = nothing, nothing, nothing

(nothing, nothing, nothing)

In [74]:
site_number = [x for x in 1:length(result_inner)];
result_inner2 = real(result_inner);
result_norm2 = real(result_norm);
result_error2 = real(result_error);

In [75]:
error_bar = nothing

In [76]:
error_bar = [sqrt(x) for x in result_error2[1:end]];

In [86]:
scatter(site_number, 
    result_norm2.± error_bar, 
    xlim=(0,35),xticks=((0:5:35)), 
    ylim=(0.34,0.49), 
    yticks=((0.34:0.01:0.49)),   
    markersize=2,
    xlabel="Site Number",
    ylabel="Inner Product",
    label="m₀=0.5,G²=1.5",
    title="Overlap")
# savefig("m05_2.svg")

"D:\\Amin\\GN\\G^2\\1.5\\m05_2.svg"

In [87]:
# writedlm("m05_inner.txt",result_inner2)
# writedlm("m05_norm.txt",result_norm2)
# writedlm("m05_error.txt",result_error2)